# Performance counters for SVD vs FJLT (No data save)

In [ ]:
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
sys.path.append('../../../utils')
import gc
from time import perf_counter
from TurboFJLT import *
from TurboFJLT_helpers import FJLT, TurboHDF5Reader

In [ ]:
%config InlineBackend.figure_format='retina'
plt.style.use("../../../mplstyles/paper_half.mplstyle")

In [ ]:
timing_averaging_iterations = 5

In [ ]:
datafile = "../../../data/fine_airfoil_cascade.h5"
reader = TurboHDF5Reader(datafile)
print(reader)

### Extract the data

In [ ]:
num_snapshots = 500
snapshot_sequence = list(range(num_snapshots))

In [ ]:
q_mf = reader.load_meanflow()

In [ ]:
def formQ(reader, seq_to_extract):
    num_dofs = reader.state_dim
    Q = np.zeros((num_dofs, len(list(seq_to_extract))))
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        Q[:, i] = reader.load_next()-q_mf
    return Q

In [ ]:
Q = formQ(reader, snapshot_sequence)
dt_direct = 0
for _ in range(timing_averaging_iterations):
    t0 = perf_counter()
    u, s, vh = np.linalg.svd(Q, full_matrices=False)
    t1 = perf_counter()
    dt_direct += t1-t0
dt_direct /= timing_averaging_iterations
print("Direct SVD application time: {}s".format(dt_direct))

In [ ]:
# Garbage collection for memory
del Q
del u
del s
del vh
gc.collect()

In [ ]:
def formB(reader, fjlt, seq_to_extract):
    num_dofs = reader.state_dim
    B = np.zeros((fjlt.embedding_dim, len(list(seq_to_extract))))
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        q_ss = reader.load_next()-q_mf
        B[:, i] = applyFJLT(q_ss, fjlt.P, fjlt.s, fjlt.D)
    return B

### Compute the SVD with the FJLT

In [ ]:
def fjlt_svd(reader, num_linking_snapshots, snapshots_to_extract):
    fjlt = FJLT(reader.state_dim, num_linking_snapshots, 0.01)
    B = formB(reader, fjlt, snapshots_to_extract)
    dt = 0
    for _ in range(timing_averaging_iterations):
        t0 = perf_counter()
        fjlt_u, fjlt_s, fjlt_vh = np.linalg.svd(B, full_matrices=False)
        t1 = perf_counter()
        dt += t1 - t0
    dt /= timing_averaging_iterations
    return fjlt_u, fjlt_s, fjlt_vh, dt

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
perf_counters = []
for n_sp in num_linking_snapshots:
    fjlt_u, fjlt_s, fjlt_vh, dt_perf = fjlt_svd(reader, n_sp, snapshot_sequence)
    print("FJLT ({}, snapshots) SVD application time: {}s".format(n_sp, dt_perf))
    perf_counters.append(dt_perf)
    # Garbage collection for memory
    del fjlt_u
    del fjlt_s
    del fjlt_vh
    gc.collect()

### Plot the performance counter timings

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)
ax.plot(num_linking_snapshots, perf_counters, marker="o")
ax.set_xlabel(r"N")
ax.set_ylabel(r"Time (s)")

In [ ]:
print(num_linking_snapshots)
print(perf_counters)